In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from pycaret.classification import *
from pycaret.classification import *
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
import function_script


In [2]:

if __name__ == "__main__":
    #Parameter
    target_csv  = 'abalone'#['abalone','50_50', 'fill']
    ground_truth = 'Sex' #['Sex', 'Diabetes_binary']
    Prediction_label = 'All' #['M_F', 'All', 'M', 'F', 'I'] 
    display_bool = False
    rand_seed = 12
    num_of_models = 3
    #1.1 Load ============================================================
    if target_csv == 'fill':
        df = pd.read_csv('../data/diabetes_binary_health_indicators_BRFSS2015.csv')
    if target_csv == '50_50':
        df = pd.read_csv('../data/diabetes_binary_5050split_health_indicators_BRFSS2015.csv')
    if target_csv == 'abalone':
        df = pd.read_csv('../data/abalone.csv')
    
    
    #2.1 Load Label ['M_F', 'All']  ============================================================        
    if target_csv == 'abalone' and Prediction_label == 'M_F':  #If classifing Male or Female
        df = df[df['Sex'] != 'I']
    if target_csv == 'abalone' and Prediction_label == 'M': #If classifing Male or not Male
        df['Sex'] = df['Sex'].replace(['F','I'],"N")
    if target_csv == 'abalone' and Prediction_label == 'F': #If classifing Male or not Male
        df['Sex'] = df['Sex'].replace(['M','I'],"N")
    if target_csv == 'abalone' and Prediction_label == 'I': #If classifing Male or not Male
        df['Sex'] = df['Sex'].replace(['M','F'],"N")
    
    print(df['Sex'])

  

    #3.1 Machine learning ============================================================
    #3.2 Splitting 60/20/20
   
    train_val_data, test_data = train_test_split(df, test_size=0.2, random_state=rand_seed) #split 80/20
    print(train_val_data.shape, test_data.shape)
  
    clf1 = setup(data=train_val_data,
                target=ground_truth,
                session_id=rand_seed,
                train_size=0.75,  #split 80 to 60/20
                #Preprocessing
                preprocess=True,  
                normalize=True, 
                normalize_method='zscore', # ['zscore','minmax','maxabs', 'robust']
                #Feature Engineering
                polynomial_features= True,
                polynomial_degree =2,
                fix_imbalance= True,
                fix_imbalance_method= 'smote',
                #feature_selection=True,
                #bin_numeric_features=['Rings'],
                #remove_outliers =True,
               # group_features = {'weights_group': ['Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight']},
                 
                #Others 
                fold = 10,
                use_gpu =False,
                )
    print(clf1)


0       M
1       M
2       F
3       M
4       I
       ..
4172    F
4173    M
4174    M
4175    F
4176    M
Name: Sex, Length: 4177, dtype: object
(3341, 9) (836, 9)


,Description,Value
0,Session id,12
1,Target,Sex
2,Target type,Multiclass
3,Target mapping,"F: 0, I: 1, M: 2"
4,Original data shape,"(3341, 9)"
5,Transformed data shape,"(3650, 45)"
6,Transformed train set shape,"(2814, 45)"
7,Transformed test set shape,"(836, 45)"
8,Numeric features,8
9,Preprocess,True


In [3]:
    best_model = compare_models(sort = 'Accuracy', n_select = num_of_models)
    print(best_model)
    if num_of_models != 1:
        best_model = blend_models(estimator_list=best_model, method='soft', optimize='Accuracy')




,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.5477,0.7481,0.5477,0.5409,0.5407,0.3229,0.3251,0.0120
lr,Logistic Regression,0.5405,0.7559,0.5405,0.5349,0.5253,0.3162,0.3234,0.2050
rf,Random Forest Classifier,0.5401,0.7346,0.5401,0.5354,0.5366,0.3090,0.3096,0.1140
ridge,Ridge Classifier,0.5389,0.0000,0.5389,0.5332,0.5195,0.3145,0.3240,0.0100
gbc,Gradient Boosting Classifier,0.5361,0.7385,0.5361,0.5279,0.5284,0.3044,0.3065,0.7500
et,Extra Trees Classifier,0.5329,0.7330,0.5329,0.5295,0.5302,0.2972,0.2977,0.0620
lightgbm,Light Gradient Boosting Machine,0.5289,0.7231,0.5289,0.5264,0.5257,0.2919,0.2931,2.4790
svm,SVM - Linear Kernel,0.5253,0.0000,0.5253,0.5148,0.4977,0.2951,0.3081,0.0200
ada,Ada Boost Classifier,0.5189,0.7153,0.5189,0.5121,0.5107,0.2796,0.2823,0.0640
nb,Naive Bayes,0.5182,0.7259,0.5182,0.5096,0.4823,0.2860,0.3033,0.0090


[LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                           priors=None, shrinkage=None, solver='svd',
                           store_covariance=False, tol=0.0001), LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=12, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=12, 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5737,0.7680,0.5737,0.5821,0.5740,0.3628,0.3655
1,0.5139,0.7294,0.5139,0.5007,0.5019,0.2712,0.2741
2,0.5976,0.7823,0.5976,0.5948,0.5939,0.3974,0.3990
3,0.5657,0.7591,0.5657,0.5640,0.5624,0.3500,0.3516
4,0.5777,0.7541,0.5777,0.5672,0.5685,0.3675,0.3699
5,0.5320,0.7446,0.5320,0.5208,0.5180,0.3007,0.3056
6,0.5160,0.7128,0.5160,0.5084,0.5110,0.2739,0.2746
7,0.5600,0.7466,0.5600,0.5544,0.5510,0.3436,0.3475
8,0.5920,0.7753,0.5920,0.5890,0.5821,0.3914,0.3968


In [4]:
    #tune model
    best_model = tune_model (best_model, optimize ='Accuracy') 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5817,0.7684,0.5817,0.5877,0.5817,0.3742,0.3763
1,0.5299,0.7299,0.5299,0.5190,0.5193,0.2952,0.2981
2,0.5936,0.7814,0.5936,0.5909,0.5890,0.3921,0.3944
3,0.5578,0.7588,0.5578,0.5575,0.5549,0.3383,0.3402
4,0.5737,0.7529,0.5737,0.5624,0.5643,0.3612,0.3636
5,0.5400,0.7443,0.5400,0.5297,0.5271,0.3122,0.3167
6,0.5160,0.7115,0.5160,0.5084,0.5110,0.2739,0.2746
7,0.5600,0.7464,0.5600,0.5522,0.5503,0.3433,0.3469
8,0.5840,0.7741,0.5840,0.5784,0.5757,0.3784,0.3819


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [5]:
    #Ensemble Model
    if num_of_models == 1:
        best_model = ensemble_model(
            estimator=best_model,  # the tuned model you want to ensemble
        )


In [6]:
    #3.3 Model Information
    evaluate_model(best_model)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [7]:
    final_model = finalize_model(best_model)

In [8]:
    #INFORMATION for test_data
    predictions = predict_model(final_model, data=test_data)
    print(predictions)

    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
    
    # Get the actual labels
    actual_labels = test_data[ground_truth]  # Replace 'ground_truth' with the name of your actual label column
    
    # Get the predicted labels from the predictions DataFrame
    predicted_labels = predictions['prediction_label']

  
    # Calculate various performance metrics
    accuracy = accuracy_score(actual_labels, predicted_labels)
    precision = precision_score(actual_labels, predicted_labels, average='weighted')  
    recall = recall_score(actual_labels, predicted_labels, average='weighted')       
    f1 = f1_score(actual_labels, predicted_labels, average='weighted')               
    
    # Print the performance metrics
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

    from sklearn.metrics import confusion_matrix, classification_report
    
    cm = confusion_matrix(actual_labels, predicted_labels)
    cr = classification_report(actual_labels, predicted_labels)
    
    print("\nConfusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.5813,0.7797,0.5813,0.5703,0.5741,0.3718,0.3728


      Length  Diameter  Height  Whole_weight  Shucked_weight  Viscera_weight  \
4161   0.585     0.455   0.170        0.9945          0.4255          0.2630   
599    0.560     0.445   0.195        0.9810          0.3050          0.2245   
2154   0.550     0.440   0.135        0.8790          0.3680          0.2095   
2775   0.585     0.460   0.145        0.8465          0.3390          0.1670   
846    0.540     0.420   0.140        0.8050          0.3690          0.1725   
...      ...       ...     ...           ...             ...             ...   
2948   0.635     0.500   0.190        1.2900          0.5930          0.3045   
783    0.385     0.280   0.125        0.2440          0.1020          0.0380   
46     0.470     0.370   0.120        0.5795          0.2930          0.2270   
3586   0.630     0.465   0.150        1.0315          0.4265          0.2400   
1244   0.385     0.300   0.095        0.3020          0.1520          0.0615   

      Shell_weight  Rings  Sex predicti

In [9]:
# Save your model
save_model(best_model, 'best_abalone_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Length', 'Diameter', 'Height',
                                              'Whole_weight', 'Shucked_weight',
                                              'Viscera_weight', 'Shell_weight',
                                              'Rings'],
                                     transformer=SimpleImputer(add_indicato...
                                                                       criterion='gini',
                                                                       max_depth=None,
                                                                       max_features='sqrt',
                                       